This is the Smart Factory Exercise.

In [1]:
import mdptoolbox, mdptoolbox.example
import numpy as np
from enum import Enum

Define all items, states and actions of our model. <br>
* We have three different items (WHITE, BLUE, RED)
* We have four different possible states of each warehouse field (EMPTY, WHITE, BLUE, RED)
* We have six possible actions for our agent (STORE and RESTORE in combination with each item color)

In [5]:
items = ['WHITE', 'BLUE', 'RED']
fieldStatus = ['WHITE', 'BLUE', 'RED', 'EMPTY']
actions = []
for operation in ['STORE', 'RESTORE']:
    for item in items:
        actions.append(operation + ' ' + item)

print(items)
print(fieldStatus)
print(actions)

['WHITE', 'BLUE', 'RED']
['WHITE', 'BLUE', 'RED', 'EMPTY']
['STORE WHITE', 'STORE BLUE', 'STORE RED', 'RESTORE WHITE', 'RESTORE BLUE', 'RESTORE RED']


Create all fields of the warehouse of size lenght x heigth (in our case 2 x 2)

In [6]:
def createWarehouseFields(length, height):
    warehouseFields = []
    for i in range(0, length):
        for j in range(0, height):
            warehouseFields.append((i,j))
    return warehouseFields

warehouseFields = createWarehouseFields(2, 2)
print(warehouseFields)

[(0, 0), (0, 1), (1, 0), (1, 1)]


Next create all possible states of our warehouse. 
In our case we have 4 fields with 4 different states each (EMPTY, WHITE, BLUE, RED), which results in 4^4 states.

In [20]:
# TODO dynamic state generation
def getStates(warehouseFields, fieldStatus):
    # TODO create state array   
    possibleStates = len(warehouseFields)**len(fieldStatus)
    states = np.zeros(possibleStates)
    pass

states = getStates(warehouseFields, fieldStatus)

Create a reward which fits our problem. <br>
The reward is higher if the distance our agent has to cover is lower.

In [ ]:
# TODO create rewards

Bring it all together now and create the transition and the reward matrix. <br>

In [26]:
# TODO create Transition and reward matrix
transition = np.zeros((len(actions), len(states), len(states)))

rewards = np.zeros((len(states), len(actions)))


[[0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [1. 0. 2. 0. 0. 0.]
 [0. 1. 2. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


Finally create the mdp models and evaluate the different classes

In [22]:
# try model:
#mdpWarehousePolicy = mdptoolbox.mdp.PolicyIteration(transition, rewards, 0.3)

# Run the MDP
#mdpWarehousePolicy.run()

print('PolicyIteration:')
#print(mdpWarehousePolicy.policy)
#print(mdpWarehousePolicy.V)
#print(mdpWarehousePolicy.iter)

#mdpWarehouseQ = mdptoolbox.mdp.QLearning(transition, rewards, 0.3)

# Run the MDP
#mdpWarehouseQ.run()

#print('Q learning:')
#print(mdpWarehouseQ.policy)
#print(mdpWarehouseQ.V)

PolicyIteration:
